References:
https://drive.google.com/drive/folders/1y6JWx2CpsOuka00uePe72jNgr7F9sK45?usp=sharingOriginal 

In [4]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers,optimizers
from keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pickle              # For saving models to disk and loading saved models
import os
import zipfile
import PIL
import PIL.Image
import pathlib
import cv2 as cv
import shutil

#1.0 YOLO V5 - Preparing the Dataset

## Google Drive 

Mounting,
Creating directories in local VM.
Setting Path Variables. 

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
dp = '/content/drive/MyDrive/glaiml/Dataset';
dp_train = os.path.join(dp,'Car Images/Train Images')
dp_test = os.path.join(dp,'Car Images/Test Images')
os.chdir(dp)

In [ ]:
%ls

 Annotations/  'Car Images'/  'Car names and make.csv'   __MACOSX/


In [ ]:
out_dp = '/content/drive/MyDrive/glaiml/car_data_yolo'

In [12]:
# Required only on the first run!
# Creating the directory sturcture in local VMs as per YOLO structure. 
os.chdir(out_dp)
%mkdir -p train/images test/images val/images train/labels test/labels val/labels

## DataFrame - reading from CSVs.

In [8]:
car_img_path = os.path.join(dp,'Car Images')
car_ann_path = os.path.join(dp,'Annotations')
df_train = pd.read_csv(os.path.join(car_ann_path,'Train Annotations.csv'),names=['Image_Name','Start_x','Start_y','End_x','End_y','Image_class'])
df_test = pd.read_csv(os.path.join(car_ann_path,'Test Annotation.csv'),names=['Image_Name','Start_x','Start_y','End_x','End_y','Image_class'])
df_cars_name = pd.read_csv(os.path.join(dp,'Car names and make.csv'),names=['Car Name'])
df_train.drop([df_train.index[0]],inplace=True)
df_test.drop([df_train.index[0]],inplace=True)

In [9]:
#df_cars_name["Car Name"].values.tolist()
df_cars_name.size

196

## Data PreProcessing

  1. Moving the images to the test directory
  1. Moving all the images to the train directory
  1. Creating a txt file corresponding to all the images
  1. Converting the images to the YOLO specific item.

In [ ]:
#Moving the Test Images and Train Images to the the VM's folder. Will take quite a bit of time. 
for root, dirs, files in os.walk(dp_test):
  for file in files:
    path_file = os.path.join(root,file)
    shutil.copy2(path_file,out_dp + '/test/images')

for root, dirs, files in os.walk(dp_train):
  for file in files:
    path_file = os.path.join(root,file)
    shutil.copy2(path_file,'/train/images')


In [ ]:
# YOLO requires the normalized height and width format.
def YOLO_xywh(xmin, ymin, xmax, ymax, imgw,imgh):
  xcen = float((xmin + xmax)) / 2 / imgw
  ycen = float((ymin + ymax)) / 2 / imgh
  w = float((xmax - xmin)) / imgw
  h = float((ymax - ymin)) / imgh
  return xcen, ycen, w, h

#Reference https://github.com/tzutalin/labelImg/blob/master/libs/yolo_io.py


In [ ]:
# data["column_name"] = data["column_name"].apply(lambda x: x.str.replace("characters_need_to_replace", "new_characters"))

df_train['Image_Path'] = df_train['Image_Name'].apply(lambda x:os.path.join(out_dp + '/train/images',x))
df_train['Image_Path'] = df_train['Image_Name'].apply(lambda x:os.path.join(out_dp + '/train/images',x))

In [ ]:
df_train['Image_Path']

1       /content/cardata/train/images/00001.jpg
2       /content/cardata/train/images/00002.jpg
3       /content/cardata/train/images/00003.jpg
4       /content/cardata/train/images/00004.jpg
5       /content/cardata/train/images/00005.jpg
                         ...                   
8140    /content/cardata/train/images/08140.jpg
8141    /content/cardata/train/images/08141.jpg
8142    /content/cardata/train/images/08142.jpg
8143    /content/cardata/train/images/08143.jpg
8144    /content/cardata/train/images/08144.jpg
Name: Image_Path, Length: 8144, dtype: object

In [ ]:
img = cv.imread('/content/cardata/train/images/00128.jpg')
img.shape

(329, 438, 3)

In [ ]:
#Populating dataset with the original code: 
def get_image_dims(df,col):
    height = []
    width = []
    for img_path in df[col].values:
        img = cv.imread(img_path)
        height.append(int(img.shape[0]))
        width.append(int(img.shape[1]))
    return height,width

df_train['Orig_img_height'],df_train['Orig_img_width'] =  get_image_dims(df_train,'Image_Path')
df_test['Orig_img_height'],df_test['Orig_img_width'] =  get_image_dims(df_test,'Image_Path')

In [ ]:
#Iterate over the rows
# TODO: Just 100 are done. We need to do the remaining ones.
for index, row in df_train.iterrows():
    print(row['Start_x'], row['Start_y'])
    x, y, w, h = YOLO_xywh(row['Start_x'],	row['Start_y'],	row['End_x'],	row['End_y'],row['Orig_img_width'], row['Orig_img_height'])
    with open(row['Image_Path'], "w") as text_file:
      print(f"{row['Image_class']} {x} {y} {w} {h}", file=text_file)

for index, row in df_test.iterrows():
    print(row['Start_x'], row['Start_y'])
    x, y, w, h = YOLO_xywh(row['Start_x'],	row['Start_y'],	row['End_x'],	row['End_y'],row['Orig_img_width'], row['Orig_img_height'])
    with open(row['Image_Path'], "w") as text_file:
      print(f"{row['Image_class']} {x} {y} {w} {h}", file=text_file)

## Installing the YOLOV5 

1. Download YOLOV5 from github. 
2. Install the requirements document
3. Configure the requirement yaml.

In [ ]:
%%capture
!git clone https://github.com/ultralytics/yolov5.git
%cd /content/yolov5
!pip install -r requirements.txt

In [ ]:
!touch carData.yaml

In [ ]:
%cat carData.yaml

In [ ]:
%%writefile carData.yaml

train: /content/cardata/train/images
test: /content/cardata/test/images

# number of classes
nc: 196

# class names
names: ['AM General Hummer SUV 2000', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura Integra Type R 2001', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi A5 Coupe 2012', 'Audi TTS Coupe 2012', 'Audi R8 Coupe 2012', 'Audi V8 Sedan 1994', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi TT Hatchback 2011', 'Audi S6 Sedan 2011', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S4 Sedan 2012', 'Audi S4 Sedan 2007', 'Audi TT RS Coupe 2012', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW 1 Series Convertible 2012', 'BMW 1 Series Coupe 2012', 'BMW 3 Series Sedan 2012', 'BMW 3 Series Wagon 2012', 'BMW 6 Series Convertible 2007', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW Z4 Convertible 2012', 'Bentley Continental Supersports Conv. Convertible 2012', 'Bentley Arnage Sedan 2009', 'Bentley Mulsanne Sedan 2011', 'Bentley Continental GT Coupe 2012', 'Bentley Continental GT Coupe 2007', 'Bentley Continental Flying Spur Sedan 2007', 'Bugatti Veyron 16.4 Convertible 2009', 'Bugatti Veyron 16.4 Coupe 2009', 'Buick Regal GS 2012', 'Buick Rainier SUV 2007', 'Buick Verano Sedan 2012', 'Buick Enclave SUV 2012', 'Cadillac CTS-V Sedan 2012', 'Cadillac SRX SUV 2012', 'Cadillac Escalade EXT Crew Cab 2007', 'Chevrolet Silverado 1500 Hybrid Crew Cab 2012', 'Chevrolet Corvette Convertible 2012', 'Chevrolet Corvette ZR1 2012', 'Chevrolet Corvette Ron Fellows Edition Z06 2007', 'Chevrolet Traverse SUV 2012', 'Chevrolet Camaro Convertible 2012', 'Chevrolet HHR SS 2010', 'Chevrolet Impala Sedan 2007', 'Chevrolet Tahoe Hybrid SUV 2012', 'Chevrolet Sonic Sedan 2012', 'Chevrolet Express Cargo Van 2007', 'Chevrolet Avalanche Crew Cab 2012', 'Chevrolet Cobalt SS 2010', 'Chevrolet Malibu Hybrid Sedan 2010', 'Chevrolet TrailBlazer SS 2009', 'Chevrolet Silverado 2500HD Regular Cab 2012', 'Chevrolet Silverado 1500 Classic Extended Cab 2007', 'Chevrolet Express Van 2007', 'Chevrolet Monte Carlo Coupe 2007', 'Chevrolet Malibu Sedan 2007', 'Chevrolet Silverado 1500 Extended Cab 2012', 'Chevrolet Silverado 1500 Regular Cab 2012', 'Chrysler Aspen SUV 2009', 'Chrysler Sebring Convertible 2010', 'Chrysler Town and Country Minivan 2012', 'Chrysler 300 SRT-8 2010', 'Chrysler Crossfire Convertible 2008', 'Chrysler PT Cruiser Convertible 2008', 'Daewoo Nubira Wagon 2002', 'Dodge Caliber Wagon 2012', 'Dodge Caliber Wagon 2007', 'Dodge Caravan Minivan 1997', 'Dodge Ram Pickup 3500 Crew Cab 2010', 'Dodge Ram Pickup 3500 Quad Cab 2009', 'Dodge Sprinter Cargo Van 2009', 'Dodge Journey SUV 2012', 'Dodge Dakota Crew Cab 2010', 'Dodge Dakota Club Cab 2007', 'Dodge Magnum Wagon 2008', 'Dodge Challenger SRT8 2011', 'Dodge Durango SUV 2012', 'Dodge Durango SUV 2007', 'Dodge Charger Sedan 2012', 'Dodge Charger SRT-8 2009', 'Eagle Talon Hatchback 1998', 'FIAT 500 Abarth 2012', 'FIAT 500 Convertible 2012', 'Ferrari FF Coupe 2012', 'Ferrari California Convertible 2012', 'Ferrari 458 Italia Convertible 2012', 'Ferrari 458 Italia Coupe 2012', 'Fisker Karma Sedan 2012', 'Ford F-450 Super Duty Crew Cab 2012', 'Ford Mustang Convertible 2007', 'Ford Freestar Minivan 2007', 'Ford Expedition EL SUV 2009', 'Ford Edge SUV 2012', 'Ford Ranger SuperCab 2011', 'Ford GT Coupe 2006', 'Ford F-150 Regular Cab 2012', 'Ford F-150 Regular Cab 2007', 'Ford Focus Sedan 2007', 'Ford E-Series Wagon Van 2012', 'Ford Fiesta Sedan 2012', 'GMC Terrain SUV 2012', 'GMC Savana Van 2012', 'GMC Yukon Hybrid SUV 2012', 'GMC Acadia SUV 2012', 'GMC Canyon Extended Cab 2012', 'Geo Metro Convertible 1993', 'HUMMER H3T Crew Cab 2010', 'HUMMER H2 SUT Crew Cab 2009', 'Honda Odyssey Minivan 2012', 'Honda Odyssey Minivan 2007', 'Honda Accord Coupe 2012', 'Honda Accord Sedan 2012', 'Hyundai Veloster Hatchback 2012', 'Hyundai Santa Fe SUV 2012', 'Hyundai Tucson SUV 2012', 'Hyundai Veracruz SUV 2012', 'Hyundai Sonata Hybrid Sedan 2012', 'Hyundai Elantra Sedan 2007', 'Hyundai Accent Sedan 2012', 'Hyundai Genesis Sedan 2012', 'Hyundai Sonata Sedan 2012', 'Hyundai Elantra Touring Hatchback 2012', 'Hyundai Azera Sedan 2012', 'Infiniti G Coupe IPL 2012', 'Infiniti QX56 SUV 2011', 'Isuzu Ascender SUV 2008', 'Jaguar XK XKR 2012', 'Jeep Patriot SUV 2012', 'Jeep Wrangler SUV 2012', 'Jeep Liberty SUV 2012', 'Jeep Grand Cherokee SUV 2012', 'Jeep Compass SUV 2012', 'Lamborghini Reventon Coupe 2008', 'Lamborghini Aventador Coupe 2012', 'Lamborghini Gallardo LP 570-4 Superleggera 2012', 'Lamborghini Diablo Coupe 2001', 'Land Rover Range Rover SUV 2012', 'Land Rover LR2 SUV 2012', 'Lincoln Town Car Sedan 2011', 'MINI Cooper Roadster Convertible 2012', 'Maybach Landaulet Convertible 2012', 'Mazda Tribute SUV 2011', 'McLaren MP4-12C Coupe 2012', 'Mercedes-Benz 300-Class Convertible 1993', 'Mercedes-Benz C-Class Sedan 2012', 'Mercedes-Benz SL-Class Coupe 2009', 'Mercedes-Benz E-Class Sedan 2012', 'Mercedes-Benz S-Class Sedan 2012', 'Mercedes-Benz Sprinter Van 2012', 'Mitsubishi Lancer Sedan 2012', 'Nissan Leaf Hatchback 2012', 'Nissan NV Passenger Van 2012', 'Nissan Juke Hatchback 2012', 'Nissan 240SX Coupe 1998', 'Plymouth Neon Coupe 1999', 'Porsche Panamera Sedan 2012', 'Ram C/V Cargo Van Minivan 2012', 'Rolls-Royce Phantom Drophead Coupe Convertible 2012', 'Rolls-Royce Ghost Sedan 2012', 'Rolls-Royce Phantom Sedan 2012', 'Scion xD Hatchback 2012', 'Spyker C8 Convertible 2009', 'Spyker C8 Coupe 2009', 'Suzuki Aerio Sedan 2007', 'Suzuki Kizashi Sedan 2012', 'Suzuki SX4 Hatchback 2012', 'Suzuki SX4 Sedan 2012', 'Tesla Model S Sedan 2012', 'Toyota Sequoia SUV 2012', 'Toyota Camry Sedan 2012', 'Toyota Corolla Sedan 2012', 'Toyota 4Runner SUV 2012', 'Volkswagen Golf Hatchback 2012', 'Volkswagen Golf Hatchback 1991', 'Volkswagen Beetle Hatchback 2012', 'Volvo C30 Hatchback 2012', 'Volvo 240 Sedan 1993', 'Volvo XC90 SUV 2007', 'smart fortwo Convertible 2012']

Overwriting carData.yaml


In [ ]:
%cat carData.yaml


train: /content/cardata/train/images
test: /content/cardata/test/images
# train: ../Road_Sign_Dataset/images/train/ 
# val:  ../Road_Sign_Dataset/images/val/
# test: ../Road_Sign_Dataset/images/test/

# number of classes
nc: 196

# class names
names: ['AM General Hummer SUV 2000', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura TL Type-S 2008', 'Acura TSX Sedan 2012', 'Acura Integra Type R 2001', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin V8 Vantage Coupe 2012', 'Aston Martin Virage Convertible 2012', 'Aston Martin Virage Coupe 2012', 'Audi RS 4 Convertible 2008', 'Audi A5 Coupe 2012', 'Audi TTS Coupe 2012', 'Audi R8 Coupe 2012', 'Audi V8 Sedan 1994', 'Audi 100 Sedan 1994', 'Audi 100 Wagon 1994', 'Audi TT Hatchback 2011', 'Audi S6 Sedan 2011', 'Audi S5 Convertible 2012', 'Audi S5 Coupe 2012', 'Audi S4 Sedan 2012', 'Audi S4 Sedan 2007', 'Audi TT RS Coupe 2012', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW 1 Series Convertible 2012', 'BMW 1 Serie

#2.0 YOLO V5 - Training / Testing the Dataset

In [ ]:
%cd /content/yolov5

/content/yolov5


In [ ]:
!python train.py --img 160 --cfg yolov5s.yaml --hyp hyp.scratch.yaml --batch 32 --epochs 100 --data carData.yaml --weights yolov5s.pt --workers 24 --name yolo_car_classification

In [ ]:
!python test.py --weights runs/train/yolo_car_classification/weights/best.pt --data carData.yaml --task test --name yolo_car_class_det
